In [ ]:
import mlcroissant as mlc
import hashlib

## Try to use mlcroissant to transform row of csv into Croissant JSON-LD

In [21]:
with open("twitterVideo.csv", "rb") as f:
    sha256 = hashlib.sha256(f.read()).hexdigest()

print(sha256)

cc118fd404fd3dcd5b165ee838d1e284f8482d92d5bc6c6abe356edef92f6bf6


Hand make .json file.

Check validity

In [ ]:
ds = mlc.Dataset("croissant.json")
print(ds.metadata.to_json())
for r in ds.records("tweets"):
    print(r)


Does not work. mlcroissant is unable to deal with arbitary url and construct a meta data from it, it mostly uses for data repositories including HuggingFace, Kaggle, and OpenML.

## Try to construct the format in loop

In [1]:
import pandas as pd
import json
import os
import datetime

In [2]:
# read csv
df = pd.read_csv('twitterVideo.csv')

In [ ]:
df.head()

In [ ]:
# extract useful information only
df.iloc[:,0:9]

In [ ]:
df = df.iloc[:,0:9]
df

In [27]:
# helper function
def Croissant_JSON_formating(row):
    status_id = row['twitterlink'].split('/')[-1]

    croissant_JSON = {
        "@context": {
            "@language": "en",
            "@vocab": "https://schema.org/",
            "cr": "http://mlcommons.org/croissant/1.0",
            "sc": "https://schema.org/"
        },
        "@type": "sc:Dataset",
        "name": f"Tweet-ID-{status_id}",
        "description": f"A single social media video post(Tweet ID: {status_id}) by the user, structured as a Croissant Dataset.",
        "url": row['twitterlink'],
        "conformsTo": "http://mlcommons.org/croissant/1.0",
        "datePublished": row['postdatetime'],
        "creator": {
            "@type": "sc:Organization",
            "name": "Social Media Manually Collect Source"
        },
        "distribution": [
            {
            "@type": "cr:FileObject",
            "name": f"source_tweet_url_{status_id}",
            "encodingFormat": "text/html",
            "contentUrl": row['twitterlink']
            }
        ],
        
        "recordSet": [
            {
            "@type": "cr:RecordSet",
            "name": "tweet_data_record",
            "description": "The specific data fields for this tweet.",
            

            "field": [
                {
                "@type": "cr:Field",
                "name": "internal_id",
                "description": "Internal record ID from the source CSV.",
                "data_type": "sc:Integer",
                "sc:value": int(row['id'])
                },
                {
                "@type": "cr:Field",
                "name": "tweet_text",
                "description": "The full text/caption of the tweet.",
                "data_type": "sc:Text",
                "sc:value": row['caption']
                },
                {
                "@type": "cr:Field",
                "name": "likes_count",
                "description": "Number of likes/favorites.",
                "data_type": "sc:Integer",
                "sc:value": int(row['likes'])
                },
                {
                "@type": "cr:Field",
                "name": "shares_count",
                "description": "Number of shares/reposts.",
                "data_type": "sc:Integer",
                "sc:value": int(row['shares'])
                },
                {
                "@type": "cr:Field",
                "name": "post_timestamp",
                "description": "When the tweet was posted (ISO 8601).",
                "data_type": "sc:DateTime",
                "sc:value": row['postdatetime']
                },
                {
                "@type": "cr:Field",
                "name": "collection_timestamp",
                "description": "When this data was collected (ISO 8601).",
                "data_type": "sc:DateTime",
                "sc:value": row['collecttime']
                },
                {
                "@type": "cr:Field",
                "name": "video_transcript_path",
                "description": "Path to the video transcription file.",
                "data_type": "sc:Text",
                "sc:value": row['videotranscriptionpath'] if pd.notna(row['videotranscriptionpath']) else ""
                }
            ]
            }
        ]
        }
    return croissant_JSON

In [22]:
# Create folder for croissant_tweet_metadata
OUTPUT_DIR = "croissant_tweet_metadata"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# loop
for index, row in df.iterrows():
    # if index+1 == 10:
    #     break

    croissant_json = Croissant_JSON_formating(row)
    # print(croissant_json)
    
    # save JSON-LD file
    file_path = os.path.join(OUTPUT_DIR, f"tweet_metadata_{index+1}.json")
    with open(file_path, 'w') as f:
        json.dump(croissant_json, f, indent=2)
    print(f"{file_path} generated")

NOTE: transcription path will only work when current directory is at `twitterVideo`

Package above function
## Test package

In [29]:
import croissant_transformer

In [ ]:
croissant_transformer.croissant_transform("twitterVideo.csv")